<a href="https://colab.research.google.com/github/minakhan01/PAL_RL/blob/main/CPC_Computer_Usage_Train_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Drive

In [ ]:
!pip install -U -q PyDrive
!pip install httplib2==0.15.0
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pydrive.files import GoogleDriveFileList
from google.colab import auth
from oauth2client.client import GoogleCredentials

from getpass import getpass
import urllib

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from google.colab import drive as gdrive 
gdrive.mount('/content/gdrive')

     |████████████████████████████████| 102kB 6.2MB/s 
  Found existing installation: httplib2 0.17.4
    Uninstalling httplib2-0.17.4:
      Successfully uninstalled httplib2-0.17.4
Mounted at /content/gdrive


## Load data from Drive

In [ ]:
def get_folder_id(drive, parent_folder_id, folder_name):
  """ 
	Check if destination folder exists and return it's ID
	"""
  folder_exists = False

  # Auto-iterate through all files in the parent folder.
  file_list = GoogleDriveFileList()
  try:
    file_list = drive.ListFile(
        {'q': "'{0}' in parents and trashed=false".format(parent_folder_id)}
        ).GetList()
	# Exit if the parent folder doesn't exist
  except googleapiclient.errors.HttpError as err:
  # Parse error message
    message = ast.literal_eval(err.content)['error']['message']
    if message == 'File not found: ':
      exit(1)
  # Exit with stacktrace in case of other error
    else:
      raise

	# Find the the destination folder in the parent folder's files
  for file1 in file_list:
    if file1['title'] == folder_name:
      folder_exists = True
      return file1['id'], folder_exists

  return None, False

In [ ]:
experiment_id = "MinaData"

folder_name = experiment_id

# Change parentid to match that of experiments root folder in gdrive
parentid = '1fakqJ81EcE65gUtVfFWBagQfGEI9IC1j'

# Initialize sepcific experiment folder in drive
folderid, folder_exists = get_folder_id(drive, parentid, folder_name)


## Read file and redact

In [ ]:
!git clone https://github.com/minakhan01/ActivityWatchRedact

Cloning into 'ActivityWatchRedact'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 71 (delta 24), reused 64 (delta 17), pack-reused 0
Unpacking objects: 100% (71/71), done.


In [ ]:
%cd ActivityWatchRedact
!pwd
!pip install -r requirements.txt

/content/ActivityWatchRedact
/content/ActivityWatchRedact
     |████████████████████████████████| 184kB 10.1MB/s 
     |████████████████████████████████| 122kB 14.1MB/s 
     |████████████████████████████████| 92kB 8.3MB/s 
     |████████████████████████████████| 829kB 14.5MB/s 
     |████████████████████████████████| 5.6MB 26.3MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 307kB 53.6MB/s 
     |████████████████████████████████| 245kB 48.0MB/s 
     |████████████████████████████████| 1.4MB 58.2MB/s 
     |████████████████████████████████| 2.6MB 55.9MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 3.5MB 29.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 174kB 56.6MB/s 
     |████████████████████████████████|

In [ ]:
import json
import io
from redact_data import *
from redact_data import _write_file, _load_data, redact_data_basic, redact_data_custom, set_timezone

def get_files_dataset(drive, folderid):
  file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folderid)}).GetList()
  for i, file1 in enumerate(sorted(file_list, key = lambda x: x['title']), start=1):
    print(file1['title'])
    file1.GetContentFile(file1['title'])
    # yield file1['title']

def redact_file(input_file, output_file):

    data, bucket_key_dict = _load_data(input_file)
    redacted_data, unique_urls, unique_app_names = redact_data_basic(data, bucket_key_dict)

    url_blacklist = set(["docs.google.com", "www.deeplearningbook.org"])
    app_blacklist = set(["Finder"])

    user_selected_timezone = 'America/New_York'

    # Remove apps and urls from user provided black list
    redacted_data = redact_data_custom(redacted_data, url_blacklist, app_blacklist, 3)
    
    # set user selected timezone
    redacted_data = set_timezone(redacted_data, user_selected_timezone)

    _write_file(output_file, redacted_data)

get_files_dataset(drive, folderid)
filepath = "aw-buckets-export.json"
output_file = "redacted-data.json"
redact_file(filepath, output_file)
redacted_data, bucket_key_dict = _load_data(output_file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
DEBUG - URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%271Lt5bexsHSnEM7UrC9S8ZT8H9r07IBJnd%27+in+parents+and+trashed%3Dfalse&maxResults=1000&alt=json


aw-buckets-export.json
redacted_data (1).json


INFO - AFK data successfully redacted (basic)...
INFO - Web data successfully redacted (basic)...
INFO - Window data successfully redacted (basic)...
INFO - Stopwatch data successfully redacted (basic)...
INFO - Web data successfully redacted (user selected)...
INFO - Window data successfully redacted (user selected)...
INFO - Redacted data written to redacted-data.json


## Process Data

In [ ]:
import numpy as np
from datetime import datetime

print("bucket_key_dict", bucket_key_dict)
print(type(redacted_data))
# for key in redacted_data["aw-watcher-afk"].keys():
afk = redacted_data["aw-watcher-afk"]
window = redacted_data["aw-watcher-window"]
browser = redacted_data["aw-watcher-web"]
stopwatch = redacted_data["aw-watcher-stopwatch"]
print(redacted_data.keys())
# print(key)
print("window", window["events"][0])
print("afk", afk["events"][0])
print("browser", browser["events"][0])
print("window", len(window["events"]))
print("afk", len(afk["events"]))
print("browser", len(browser["events"]))

# convert to numpy array
window_vocab = {}
for event in window["events"]:
  app_name = event["data"]["app"]
  if app_name not in window_vocab:
    window_vocab[app_name] = 0
  window_vocab[app_name] += 1
print("window_vocab", len(window_vocab))

url_vocab = {}
for event in browser["events"]:
  url_name = event["data"]["url"]
  if url_name not in url_vocab:
    url_vocab[url_name] = 0
  url_vocab[url_name] += 1
print("url_vocab", len(url_vocab))

afk_vocab = {}
for event in afk["events"]:
  status_name = event["data"]["status"]
  if status_name not in afk_vocab:
    afk_vocab[status_name] = 0
  afk_vocab[status_name] += 1
print("afk_vocab", len(afk_vocab))

filtered_url_vocab = [url for url in url_vocab.keys() if url_vocab[url] > 10] 
print(len(filtered_url_vocab))

sorted_window_vocab = sorted(window_vocab.items(), key= lambda x: x[1], reverse=True)
window_word_to_id = {sorted_window_vocab[i][0]: i for i in range(len(sorted_window_vocab))}
print("window_word_to_id", window_word_to_id)
print(sorted_window_vocab)

sorted_url_vocab = sorted(url_vocab.items(), key= lambda x: x[1], reverse=True)
url_word_to_id = {sorted_url_vocab[i][0]: i for i in range(len(sorted_url_vocab))}

window_data = []
for event in window["events"]:
  # print("event", event)
  app_id = window_word_to_id[event["data"]["app"]] 
  time = event["timestamp"]
  duration = event["duration"]
  # print("duration: {}, event {}".format(duration, event))
  date_time_obj = datetime.fromisoformat(time)
  timestamp = date_time_obj.timestamp()
  item = [timestamp, duration, 0, app_id]
  # window {'timestamp': '2021-04-08T01:44:56.409000+00:00', 'duration': 228.513, 'data': {'app': 'Google Chrome'}}
  window_data.append(item)
# print("window_vocab", window_vocab)
# print("window_vocab shape", np.array(window_data).shape)

url_data = []
for event in browser["events"]:
  # print("event", event)
  url_id = url_word_to_id[event["data"]["url"]]
  tab_count = event["data"]["tabCount"]
  audible = event["data"]["audible"]                                               
  incognito = event["data"]["incognito"]                                               
  time = event["timestamp"]
  date_time_obj = datetime.fromisoformat(time)
  timestamp = date_time_obj.timestamp()
  duration = event["duration"]
  item = [timestamp, duration, 1, url_id]
  # item = np.array([timestamp, duration, 1, url_id, audible, incognito, tab_count])
  # window {'timestamp': '2021-04-08T01:44:56.409000+00:00', 'duration': 228.513, 'data': {'app': 'Google Chrome'}}
  url_data.append(item)
# print(np.array(url_data).shape)

afk_data = []
for event in afk["events"]:
  # print("event", event)
  afk_id = event["data"]["status"] == "afk"                                             
  time = event["timestamp"]
  date_time_obj = datetime.fromisoformat(time)
  timestamp = date_time_obj.timestamp()
  duration = event["duration"]
  item = [timestamp, duration, 2, afk_id]
  afk_data.append(item)
# print(np.array(afk_data).shape)


# print("stopwatch", stopwatch["events"][0])
  # print(type(afk[key]))
  # print(afk["events"][0])
# "id", "created", "name", "type", "client", "hostname", "events"
# "timestamp", "duration", "events", "current_window", "client"
# files_ds = tf.data.Dataset.from_generator(
#     lambda: get_files_dataset(drive, folderid),
#     output_signature=tf.TensorSpec(shape=(), dtype=tf.string))

# files_ds = files_ds.repeat().shuffle(10)

# tf.io.decode_json_example

bucket_key_dict {'afk': 'aw-watcher-afk', 'desktop': 'aw-watcher-window', 'browser': 'aw-watcher-web', 'stopwatch': None}
<class 'dict'>
dict_keys(['aw-watcher-window', 'aw-watcher-afk', 'aw-watcher-web', 'aw-watcher-stopwatch'])
window {'timestamp': '2021-04-08T01:44:56.409000+00:00', 'duration': 228.513, 'data': {'app': 'Google Chrome'}}
afk {'timestamp': '2021-04-08T01:10:45.510000+00:00', 'duration': 2280.741, 'data': {'status': 'not-afk'}}
browser {'timestamp': '2021-04-09T07:31:05.511000+00:00', 'duration': 0.0, 'data': {'url': 'colab.research.google.com', 'audible': False, 'incognito': False, 'tabCount': 2}}
window 2023
afk 624
browser 458173
window_vocab 12
url_vocab 1875
afk_vocab 2
539
window_word_to_id {'Google Chrome': 0, 'WhatsApp': 1, 'Preview': 2, 'zoom.us': 3, 'redacted_app_5': 4, 'Notes': 5, 'Sublime Text': 6, 'Photo Booth': 7, 'System Preferences': 8, 'TextEdit': 9, 'Archive Utility': 10, 'unknown': 11}
[('Google Chrome', 1004), ('WhatsApp', 463), ('Preview', 224), ('

In [ ]:
all_data = np.concatenate([window_data, url_data, afk_data], axis=0)
print(all_data.shape)
print(all_data[0])
py_sorted_data = sorted(all_data, key= lambda x: x[0])
sorted_data = np.array(py_sorted_data, dtype=np.int64)
# sorted_data = all_data
zero_count = 0
zero_window_count = 0
zero_url_count = 0
zero_afk_count = 0

for row in sorted_data:
  if row[1] == 0:
    zero_count += 1 
    if row[2] == 0:
      zero_window_count += 1
    if row[2] == 1:
      zero_url_count += 1
    if row[2] == 2:
      zero_afk_count += 1

print("count", zero_count, zero_window_count, zero_url_count, zero_afk_count)
print("", min(sorted_data[:,1]))
print("sorted", sorted_data[100000:100010])
print("window", window_data[:10])
print("url", url_data[:10])
print("afk", afk_data[:10])

(460820, 4)
[1.6178463e+09 2.2851300e+02 0.0000000e+00 0.0000000e+00]
count 336401 156 336221 24
 0
sorted [[1608914040         40          1          2]
 [1608914099          0          1         18]
 [1608914100         60          1          2]
 [1608914164          0          1         18]
 [1608914180         20          1          2]
 [1608914211          1          1          0]
 [1608914213          0          1          3]
 [1608914220          0          1          2]
 [1608914221          0          1          3]
 [1608914221          9          1          0]]
window [[1617846296.409, 228.513, 0, 0], [1617846270.616, 24.646, 0, 2], [1617846043.426, 225.988, 0, 0], [1617846030.499, 11.767, 0, 2], [1617844812.625, 1216.701, 0, 0], [1617844792.655, 18.781, 0, 2], [1617843205.909, 1585.57, 0, 0], [1617843185.198, 19.512, 0, 1], [1617834994.074, 8189.974, 0, 0], [1617834989.451, 3.472, 0, 1]]
url [[1617953465.511, 0.0, 1, 6], [1617953449.376, 0.0, 1, 96], [1617953449.372, 0.003, 

In [ ]:
import tensorflow as tf
max_afk_vocab_size = 2
max_window_vocab_size = 100
max_url_vocab_size = 5000

combined_data = []
for row in sorted_data:
  if row[2] == 0: # window
    rename_vocab = row[3] + max_afk_vocab_size + 1
  elif row[2] == 1: # url
    rename_vocab = row[3] + max_afk_vocab_size + max_window_vocab_size + 1 
  elif row[2] == 2:
    rename_vocab = row[3] + 1
  else:
    raise
  if row[3] == 1874:
    print("rename_vocab", rename_vocab, row[3], row[2])
  combined_data.append([row[0], row[1], rename_vocab])
np_combined = np.array(combined_data, dtype= np.int64)

DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


rename_vocab 1977 1874 1


In [ ]:
print(sorted_data.shape)
print(np_combined.shape)
print("max", np.max(np_combined[:,2]))
print(len(url_vocab))
print(len(window_vocab))
np_url = np.array(url_data)
print("np_url", np.max(np_url[:,3]))
np_all = np.array(sorted_data)
print("np_all shape", np_all.shape)
print("np_all", np.max(np_all[:,3]))

(460820, 4)
(460820, 3)
max 1977
1875
12
np_url 1874.0
np_all shape (460820, 4)
np_all 1874


In [ ]:
# print(np.histogram(np_combined[:,1], range(1000)))
# print(np.histogram(np.array(url_data)[:,1], range(100)))
# print(np.histogram(np.array(window_data)[:,1], range(100)))
print(url_data[:10])
print(sorted_url_vocab[6])
print(sorted_url_vocab[96])
print(sorted_url_vocab[12])
(np_combined[:,1] > 1).mean()

[[1617953465.511, 0.0, 1, 6], [1617953449.376, 0.0, 1, 96], [1617953449.372, 0.003, 1, 12], [1617953446.708, 2.663, 1, 10], [1617953446.707, 0.0, 1, 15], [1617953445.506, 0.0, 1, 6], [1617953440.44, 0.0, 1, 15], [1617953440.439, 0.0, 1, 2], [1617953433.76, 0.0, 1, 0], [1617953433.302, 0.0, 1, 2]]
('colab.research.google.com', 24312)
('localhost:5600', 156)
('github.com', 7350)


0.2465821795928996

### tfds

In [ ]:
import tensorflow_datasets as tfds

# fix preprocessing: use window instead of batch
timestamps = np_combined[:, 0]
# timestamps = [t1, t2, ..]
# delays = [t2-t1, t3-t2, ..., t[n]-t[n-1]]
# [t2, t3, t4, ..., tn] - [t1, t2, ..., t[n-1] ]
# timestamps[1:] - timestamps[:-1]
delays = timestamps[1:] - timestamps[:-1]
zero = np.zeros([1])
all_delay = np.concatenate([zero, delays], axis = 0) 
delay_added = np.concatenate([np_combined, all_delay[:,None]], axis = 1)
tf_data_list = tf.data.Dataset.from_tensor_slices(delay_added).batch(64, drop_remainder=True).repeat().shuffle(1600).batch(16, drop_remainder=True)
# train_dataset = tf_data
tf_data = tf_data_list.map(lambda x: {"activity_obs": tf.cast(x[:,:-1,2], tf.int32), 
                                      "duration_obs": tf.cast(x[:,:-1,1], tf.int32),
                                      "time_obs": tf.cast(x[:,:-1,0], tf.int32),
                                      "delay_obs": tf.cast(x[:,:-1,3], tf.int32),
                                      "activity_target": tf.cast(x[:,1:,2], tf.int32),
                                      "duration_target": tf.cast(x[:,1:,1], tf.int32),
                                      "delay_target": tf.cast(x[:,1:,3], tf.int32)})
# tf_data = tf_data_list.map(lambda x: {"obs": tf.cast(x[:,:-1,2], tf.int32), 
                                      # "target": tf.cast(x[:,1:,2], tf.int32)})
# print(next(iter(tf_data)).shape)
train_dataset = iter(tfds.as_numpy(tf_data))

## Model Code

In [ ]:
!pip install git+https://github.com/deepmind/dm-haiku
!pip install optax

  Cloning https://github.com/deepmind/dm-haiku to /tmp/pip-req-build-9hfc02g1
  Running command git clone -q https://github.com/deepmind/dm-haiku /tmp/pip-req-build-9hfc02g1
  Created wheel for dm-haiku: filename=dm_haiku-0.0.5.dev0-cp37-none-any.whl size=545835 sha256=1c7bd147f88adfc7d61c5fec43f9a436a44754b19f68134bf34ee680725b4a0d
  Stored in directory: /tmp/pip-ephem-wheel-cache-hb_dsh4c/wheels/97/0f/e9/17f34e377f8d4060fa88a7e82bee5d8afbf7972384768a5499
Successfully built dm-haiku
  Found existing installation: tabulate 0.8.3
    Uninstalling tabulate-0.8.3:
      Successfully uninstalled tabulate-0.8.3
     |████████████████████████████████| 102kB 5.8MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 


In [ ]:
# for i in range(1, len(np_combined)):
#   if np_combined[i-1, 1] == 0 or np_combined[i, 1] == 0:
#     continue
#   diff_time = np_combined[i,0] - np_combined[i-1,0]
#   assert diff_time > np_combined[i-1,1] , (diff_time, np_combined[i-1], np_combined[i])

print(sorted_url_vocab[111-102])
print(sorted_url_vocab[113-102])

('www.amazon.com', 13914)
('redacted_url_4', 11067)


In [ ]:
from datetime import datetime

def event_to_tick(events, delta_time, max_simul_events):
  ticks = []
  current_events = []
  start_time = (events[0][0] // delta_time ) * delta_time
  current_time = start_time
  finished = 0
  num_feature = len(events[0])
  while True:
    for event in events[finished:]:
      if event[0] <= current_time:
        current_events.append(event)
        finished += 1
      else:
        break
    current_events_plus_padding = [x for x in current_events]
    # current_events_plus_time = [[current_time]+ event.tolist()
                                # for event in current_events]
    for j in range(max_simul_events-len(current_events)):
      current_events_plus_padding.append(np.zeros((num_feature,), dtype=np.int64))
    tick_events = np.array(current_events_plus_padding)
    assert tick_events.shape[0] == max_simul_events
    tick = dict(time=np.array([current_time]), 
                start_time=tick_events[None, :, 0],
                duration=tick_events[None, :, 1],
                event_type=tick_events[None, :, 2],
                start_time_features=get_time_features(tick_events[:, 0])[None, :],
                time_features=get_time_features(np.array([current_time]))[None, :]
                )
    # print("tick",jax.tree_map(lambda x: (x.shape, x.dtype), tick))
    yield tick
    current_time += delta_time
    current_events = [event for event in current_events if event[0] + event[1] > current_time]
    if not current_events and finished == len(events):
      break

def get_time_features(unix_times):
  return np.array([list(datetime.fromtimestamp(unix_time).timetuple()) for unix_time in unix_times])

# def add_time_features_to_tick(tick):
#   time = tick[0][0]
#   time_features = list(get_time_features_from_time(time))
#   return np.array([time_features+event.tolist() for event in tick])

time_interval = 60
max_simul_events = 64
all_ticks = list(event_to_tick(np_combined, time_interval, max_simul_events))
# tfd_ticks = tf.data.Dataset.from_tensor_slices(all_ticks)
# # batch(64, drop_remainder=True).repeat().shuffle(1600).batch(16, drop_remainder=True)

# # for ticks in all_ticks[:100]:
# #   print(ticks)
# #   print("")

# print(next(iter(tfd_ticks)))
# # print(all_ticks[:10])
# print(len(all_ticks))

In [ ]:
!pip install jax

In [ ]:
import jax 
ticks_dict = jax.tree_multimap(lambda *args: np.concatenate(args, axis=0), *all_ticks)
# print(type(ticks_dict["duration"][0]))
# print("type", type(ticks_dict))
def tree_print(tree):
  print(jax.tree_map(lambda x: (x.shape, x.dtype), tree))

# for ticks in all_ticks[:100]:
#   print(ticks)
#   print("")

# print(next(iter(tfd_ticks)))
# print(all_ticks[:10])
# print(len(all_ticks))

In [ ]:
pip install dm-tree

ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: six 1.11.0
    Uninstalling six-1.11.0:
      Successfully uninstalled six-1.11.0


In [ ]:
d_seq_len = 64
shift = 1
shuffle_buffer = 3200
batch_size = 16
import tree
def dict_batch(ds):
  tuple_ds = tf.data.Dataset.zip(tuple(tree.flatten(ds)))
  return tuple_ds.map(lambda *args: tree.unflatten_as(ds, args))

tfd_ticks_all = tf.data.Dataset.from_tensor_slices(ticks_dict)
tfd_ticks_window_ds = tfd_ticks_all.window(d_seq_len, shift=shift, drop_remainder=True)
tfd_ticks_window = tfd_ticks_window_ds.interleave(dict_batch).batch(d_seq_len, drop_remainder=True)
cardinality = tfd_ticks_window_ds.cardinality().numpy()[()]
num_train = int(0.5*cardinality)
num_test = int(0.5*cardinality)
tfd_ticks_train = tfd_ticks_window.take(num_train).repeat().shuffle(shuffle_buffer).batch(batch_size, drop_remainder=True)
tfd_ticks_test = tfd_ticks_window.skip(num_train).take(num_test).repeat().shuffle(shuffle_buffer).batch(batch_size, drop_remainder=True)
print(cardinality)
# tfd_ticks = tfd_ticks_all.batch(64, drop_remainder=True).repeat().shuffle(1600).batch(16, drop_remainder=True)
# tree_print(next(iter(tfd_ticks)))

191074


In [ ]:
np_train = tfds.as_numpy(tfd_ticks_train)
np_test = tfds.as_numpy(tfd_ticks_test)

In [ ]:
tree_print(next(iter(tfd_ticks_window)))
print("")
tree_print(next(iter(tfd_ticks_all)))

{'duration': (TensorShape([64, 64]), tf.int64), 'event_type': (TensorShape([64, 64]), tf.int64), 'start_time': (TensorShape([64, 64]), tf.int64), 'start_time_features': (TensorShape([64, 64, 9]), tf.int64), 'time': (TensorShape([64]), tf.int64), 'time_features': (TensorShape([64, 1, 9]), tf.int64)}

{'duration': (TensorShape([64]), tf.int64), 'event_type': (TensorShape([64]), tf.int64), 'start_time': (TensorShape([64]), tf.int64), 'start_time_features': (TensorShape([64, 9]), tf.int64), 'time': (TensorShape([]), tf.int64), 'time_features': (TensorShape([1, 9]), tf.int64)}


# Preprocessing

## Transformer

In [ ]:
# Copyright 2020 DeepMind Technologies Limited. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Transformer model components."""

from typing import Optional

import haiku as hk
import jax
import jax.numpy as jnp
import numpy as np


class CausalSelfAttention(hk.MultiHeadAttention):
  """Self attention with a causal mask applied."""

  def __call__(
      self,
      query: jnp.ndarray,
      key: Optional[jnp.ndarray] = None,
      value: Optional[jnp.ndarray] = None,
      mask: Optional[jnp.ndarray] = None,
  ) -> jnp.ndarray:
    key = key if key is not None else query
    value = value if value is not None else query

    seq_len = query.shape[1]
    causal_mask = np.tril(np.ones((seq_len, seq_len))[None, None, :, :])
    mask = mask * causal_mask if mask is not None else causal_mask

    return super().__call__(query, key, value, mask)

class NonCausalSelfAttention(hk.MultiHeadAttention):
  """Self attention with a non-causal mask applied."""

  def __call__(
      self,
      query: jnp.ndarray,
      key: Optional[jnp.ndarray] = None,
      value: Optional[jnp.ndarray] = None,
      mask: Optional[jnp.ndarray] = None,
  ) -> jnp.ndarray:
    key = key if key is not None else query
    value = value if value is not None else query

    seq_len = query.shape[1]
    # causal_mask = np.tril(np.ones((seq_len, seq_len)))
    # mask = mask * causal_mask if mask is not None else causal_mask

    return super().__call__(query, key, value, mask)


class DenseBlock(hk.Module):
  """A 2-layer MLP which widens then narrows the input."""

  def __init__(self,
               init_scale: float,
               widening_factor: int = 4,
               name: Optional[str] = None):
    super().__init__(name=name)
    self._init_scale = init_scale
    self._widening_factor = widening_factor

  def __call__(self, x: jnp.ndarray) -> jnp.ndarray:
    hiddens = x.shape[-1]
    initializer = hk.initializers.VarianceScaling(self._init_scale)
    x = hk.Linear(self._widening_factor * hiddens, w_init=initializer)(x)
    x = jax.nn.gelu(x)
    return hk.Linear(hiddens, w_init=initializer)(x)


class Transformer(hk.Module):
  """A transformer stack."""

  def __init__(self,
               num_heads: int,
               num_layers: int,
               dropout_rate: float,
               causal: bool,
               name: Optional[str] = None):
    super().__init__(name=name)
    self._num_layers = num_layers
    self._num_heads = num_heads
    self._dropout_rate = dropout_rate
    self._causal = causal

  def __call__(self,
               h: jnp.ndarray,
               mask: Optional[jnp.ndarray],
               is_training: bool) -> jnp.ndarray:
    """Connects the transformer.
    Args:
      h: Inputs, [B, T, H].
      mask: Padding mask, [B, T].
      is_training: Whether we're training or not.
    Returns:
      Array of shape [B, T, H].
    """
    SelfAttention = CausalSelfAttention if self._causal else NonCausalSelfAttention
    init_scale = 2. / self._num_layers
    dropout_rate = self._dropout_rate if is_training else 0.
    # if mask is not None:
    #   mask = mask[:, None, None, :]

    # Note: names chosen to approximately match those used in the GPT-2 code;
    # see https://github.com/openai/gpt-2/blob/master/src/model.py.
    for i in range(self._num_layers):
      h_norm = layer_norm(h, name=f'h{i}_ln_1')
      h_attn = SelfAttention(
          num_heads=self._num_heads,
          key_size=64,
          w_init_scale=init_scale,
          name=f'h{i}_attn')(h_norm, mask=mask)
      h_attn = hk.dropout(hk.next_rng_key(), dropout_rate, h_attn)
      h = h + h_attn
      h_norm = layer_norm(h, name=f'h{i}_ln_2')
      h_dense = DenseBlock(init_scale, name=f'h{i}_mlp')(h_norm)
      h_dense = hk.dropout(hk.next_rng_key(), dropout_rate, h_dense)
      h = h + h_dense
    h = layer_norm(h, name='ln_f')

    return h


def layer_norm(x: jnp.ndarray, name: Optional[str] = None) -> jnp.ndarray:
  """Apply a unique LayerNorm to x with default settings."""
  return hk.LayerNorm(axis=-1,
                      create_scale=True,
                      create_offset=True,
                      name=name)(x)

## CPC Architecture

In [ ]:
from typing import Dict

ArrayNest = Dict[str, jnp.ndarray]

class CPC:
  class Encoder(hk.Module):
    """Takes a dict of 3d arrays indexed by batch_size, time, num_events."""

    # def __init__(self,
    #             name: Optional[str] = None):
    #   super().__init__(name=name)

    def __call__(self, inputs: ArrayNest, is_training: bool) -> jnp.ndarray:
      """{'duration': (TensorShape([16, 64, 64]), tf.int64), 
      'event_type': (TensorShape([16, 64, 64]), tf.int64), 
      'start_time': (TensorShape([16, 64, 64]), tf.int64), 
      'start_time_features': (TensorShape([16, 64, 64, 9]), tf.int64), 
      'time': (TensorShape([16, 64]), tf.int64), 
      'time_features': (TensorShape([16, 64, 1, 9]), tf.int64)}
      """
      # inputs['duration']
      # TODO: un-hardcode vocab_size and embed_dim using init
      vocab_size = 5103
      embed_dim = 256
      event_embedding = hk.Embed(vocab_size, embed_dim)(inputs["event_type"])  # [bs, t, events, dim]
      year_embedding = hk.Embed(100, embed_dim)(inputs["time_features"][..., 0]-1970)  # [bs, t, 1, dim]
      month_embedding = hk.Embed(12, embed_dim)(inputs["time_features"][..., 1]-1)  # [bs, t, 1, dim]
      day_embedding = hk.Embed(31, embed_dim)(inputs["time_features"][..., 2]-1)  # [bs, t, 1, dim]
      hour_embedding = hk.Embed(24, embed_dim)(inputs["time_features"][..., 3])  # [bs, t, 1, dim]
      min_embedding = hk.Embed(60, embed_dim)(inputs["time_features"][..., 4])  # [bs, t, 1, dim]
      week_embedding = hk.Embed(7, embed_dim)(inputs["time_features"][..., 6])  # [bs, t, 1, dim]
      # TODO: look up numpy broadcasting
      # TODO: problem: adding time information, this might make prediction too easy
      # embedding = event_embedding + year_embedding + month_embedding + day_embedding + hour_embedding + min_embedding + week_embedding
      embedding = event_embedding
      batch_size = embedding.shape[0]
      seq_len = embedding.shape[1]
      num_events = embedding.shape[2]
      embedding_btd = jnp.reshape(
          embedding, [batch_size * seq_len, num_events, embed_dim])
      mask = jnp.greater(inputs["event_type"], 0)
      event_mask = jnp.reshape(mask, [batch_size * seq_len, num_events])
      event_mask = jnp.logical_and(event_mask[:, None, :, None], event_mask[:, None, None, :])
      # print("mask shape before any", mask.shape)
      time_mask = mask.any(axis=2)
      # print("mask shape after any", time_mask.shape)
      time_mask = jnp.logical_and(time_mask[:, None, :, None], time_mask[:, None, None, :])
      transformer = Transformer(num_heads=4, 
                                num_layers=2,
                                causal=False,  
                                dropout_rate=0.1)
      # TODO: add temporal down sampling
      output_embedding = transformer(embedding_btd, 
                                     mask=event_mask, 
                                     is_training=is_training)
      output_embedding = jnp.reshape(
          output_embedding, [batch_size, seq_len, num_events, embed_dim])
      output_embedding = output_embedding.mean(axis=2)  # [bs, t, dim]
      return output_embedding, time_mask

  class CausalTemporalEncoder(hk.Module):
    def __call__(self, z_t: jnp.ndarray, mask: jnp.ndarray, is_training: bool
                 ) -> jnp.ndarray:
      """z_t: [bs, t, dim]"""
      transformer = Transformer(num_heads=4, num_layers=2, causal=True, dropout_rate=0.1)
      # print("Before CausalTemporalEncoder mask shape", mask.shape)
      c_t = transformer(z_t, mask, is_training=is_training)
      # print("After CausalTemporalEncoder mask shape", mask.shape)
      return c_t

  class Model(hk.Module):
    def __call__(self, inputs: ArrayNest, is_training: bool) -> jnp.ndarray:
      z_t, mask = CPC.Encoder()(inputs, is_training)  # [bs, t, dim]
      c_t = CPC.CausalTemporalEncoder()(z_t, mask, is_training) # [bs, t, dim]
      # c0, c1, c2, ..., c63
      # z0, z1, z2, ..., z63
      num_steps_to_ignore = 1
      num_steps_to_predict = 10
      embed_dim = z_t.shape[-1]
      batch_size = z_t.shape[0]
      seq_len = z_t.shape[1]
      loss = 0
      mask = mask.any(axis=1)
      for k in range(num_steps_to_ignore, num_steps_to_predict):
        # example: for k = 5
        # predictors c0, c1, c2, ..., c58
        # targets    z5, z6, z7, ..., z63
        # TODO: Check masking
        targets = z_t[:, k:]
        targets_mask = mask[:, k:].any(axis=2)
        predictors = c_t[:, :-k]
        predictors_mask = mask[:, :, :-k].any(axis=1)
        target_predictor_mask = jnp.logical_and(predictors_mask, targets_mask)
        # target_predictor_mask = jnp.logical_and(
        #     target_predictor_mask[:, None, :, None], 
        #     target_predictor_mask[:, None, None, :])
        # print("before step_loss, target predictor mask shape", 
        #       target_predictor_mask.shape)
        target_predictor_mask = target_predictor_mask.reshape([-1])
        predictions = hk.Linear(embed_dim)(predictors)
        flat_targets = jnp.reshape(targets, [-1, embed_dim])
        flat_predictions = jnp.reshape(predictions, [-1, embed_dim])
        logits = jnp.dot(flat_predictions, flat_targets.T)
        log_probs = jax.nn.log_softmax(logits)
        num_items = batch_size*(seq_len-k)
        labels = jnp.arange(num_items)[...,None]
        # print("before step_loss, log_probs", log_probs.shape)
        # print("before step_loss, labels shape", labels.shape)
        # print("after step_loss, target predictor mask shape", 
        #       target_predictor_mask.shape)
        step_loss = jnp.take_along_axis(-log_probs, labels, axis=-1)[..., 0]
        # print("after step_loss, step_loss shape", step_loss.shape)
        loss += jnp.sum(step_loss * target_predictor_mask) / jnp.sum(target_predictor_mask)
      return loss

In [ ]:
def cpc_loss_fn(inputs: ArrayNest):
  return CPC.Model()(inputs, is_training=True)

In [ ]:
cpc_model = hk.transform(cpc_loss_fn)
train_iter = iter(np_train)
test_iter = iter(np_test)

In [ ]:
import optax 
import functools
from typing import Any, Mapping

class Updater:
  """A stateless abstraction around an init_fn/update_fn pair.
  This extracts some common boilerplate from the training loop.
  """

  def __init__(self, net_init, loss_fn,
               optimizer: optax.GradientTransformation):
    self._net_init = net_init
    self._loss_fn = loss_fn
    self._opt = optimizer

  @functools.partial(jax.jit, static_argnums=0)
  def init(self, master_rng, data):
    """Initializes state of the updater."""
    out_rng, init_rng = jax.random.split(master_rng)
    params = self._net_init(init_rng, data)
    opt_state = self._opt.init(params)
    out = dict(
        step=np.array(0),
        rng=out_rng,
        opt_state=opt_state,
        params=params,
    )
    return out

  @functools.partial(jax.jit, static_argnums=0)
  def update(self, state: Mapping[str, Any], data: Mapping[str, jnp.ndarray]):
    """Updates the state using some data and returns metrics."""
    rng, new_rng = jax.random.split(state['rng'])
    params = state['params']
    loss, g = jax.value_and_grad(self._loss_fn)(params, rng, data)

    updates, opt_state = self._opt.update(g, state['opt_state'])
    params = optax.apply_updates(params, updates)

    new_state = {
        'step': state['step'] + 1,
        'rng': new_rng,
        'opt_state': opt_state,
        'params': params,
    }

    metrics = {
        'step': state['step'],
        'loss': loss,
    }
    return new_state, metrics

  @functools.partial(jax.jit, static_argnums=0)
  def eval(self, state: Mapping[str, Any], data: Mapping[str, jnp.ndarray]):
    """Updates the state using some data and returns metrics."""
    rng, new_rng = jax.random.split(state['rng'])
    params = state['params']
    loss = self._loss_fn(params, rng, data)

    metrics = {
        'step': state['step'],
        'loss': loss,
    }
    return metrics

In [ ]:
import time
# TODO: training loop
grad_clip_value = 0.25
learning_rate = 2e-4

optimizer = optax.chain(
    optax.clip_by_global_norm(grad_clip_value),
    optax.adam(learning_rate, b1=0.9, b2=0.99))

updater = Updater(cpc_model.init, cpc_model.apply, optimizer)
# updater = CheckpointingUpdater(updater, checkpoint_dir)

# Initialize parameters.
logging.info('Initializing parameters...')
rng = jax.random.PRNGKey(428)
data = next(train_iter)
tree_print(data)
state = updater.init(rng, data)

LOG_EVERY = 50
EVAL_EVERY = 100
MAX_STEPS = 10**6
NUM_EVAL_ITER = 50

def eval_fn(data_iter, num_val_iter):
  eval_metrics = None
  for i in range(NUM_EVAL_ITER):
    data = next(data_iter)
    eval_metric = updater.eval(state, data)
    if eval_metrics is None:
      eval_metrics = eval_metric
    else:
      eval_metrics = {k: eval_metrics[k]+eval_metric[k] for k in eval_metrics}
  eval_metrics = {k: (eval_metrics[k]/NUM_EVAL_ITER) for k in eval_metrics}
  logging.info({k: float(v) for k, v in eval_metrics.items()})

logging.info('Starting train loop...')
prev_time = time.time()
for step in range(MAX_STEPS):
  data = next(train_iter)
  state, metrics = updater.update(state, data)
  if step % LOG_EVERY == 0:
    steps_per_sec = LOG_EVERY / (time.time() - prev_time)
    prev_time = time.time()
    metrics.update({'steps_per_sec': steps_per_sec})
    logging.info({k: float(v) for k, v in metrics.items()})
  if step % EVAL_EVERY == 0:
    print("train_iter")
    eval_fn(train_iter, NUM_EVAL_ITER)
    print("test_iter")    
    eval_fn(test_iter, NUM_EVAL_ITER)
    

INFO - Initializing parameters...
DEBUG - Initializing backend 'interpreter'
DEBUG - Initializing backend 'cpu'
DEBUG - Initializing backend 'gpu'
DEBUG - Initializing backend 'tpu'
DEBUG - Error initializing backend 'tpu': Invalid argument: TpuPlatform is not available.


{'duration': ((16, 64, 64), dtype('int64')), 'event_type': ((16, 64, 64), dtype('int64')), 'start_time': ((16, 64, 64), dtype('int64')), 'start_time_features': ((16, 64, 64, 9), dtype('int64')), 'time': ((16, 64), dtype('int64')), 'time_features': ((16, 64, 1, 9), dtype('int64'))}


DEBUG - Compiling init (139904607299536) for args (ShapedArray(uint32[2]), ShapedArray(int32[16,64,64]), ShapedArray(int32[16,64,64]), ShapedArray(int32[16,64,64]), ShapedArray(int32[16,64,64,9]), ShapedArray(int32[16,64]), ShapedArray(int32[16,64,1,9])).
INFO - Starting train loop...
DEBUG - Compiling update (139904611435216) for args (ShapedArray(int32[]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[1024]), ShapedArray(float32[256,1024]), ShapedArray(float32[256]), ShapedArray(float32[1024,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), Shaped

train_iter


DEBUG - Compiling eval (139904613808240) for args (ShapedArray(int32[]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[1024]), ShapedArray(float32[256,1024]), ShapedArray(float32[256]), ShapedArray(float32[1024,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256,256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[1024]), ShapedArray(float32[256,1024]), ShapedArray(float32[256]), ShapedArray(float32[1024,256]), ShapedArr

...

INFO - {'loss': 47.99298858642578, 'step': 27601.0}
INFO - {'loss': 36.591453552246094, 'step': 27650.0, 'steps_per_sec': 1.9177590112754868}
INFO - {'loss': 40.33780288696289, 'step': 27700.0, 'steps_per_sec': 2.9242010280003816}


train_iter


INFO - {'loss': 39.72968292236328, 'step': 27701.0}


test_iter


INFO - {'loss': 46.609676361083984, 'step': 27701.0}


In [ ]:
# tick_iter = iter(tfds.as_numpy(tfd_ticks))
# tick_iter = iter(tfd_ticks)


In [ ]:
# rng = jax.random.PRNGKey(428)
# rng, init_rng = jax.random.split(rng)
# cpc_model.init(init_rng, next(train_iter))
# cpc_model.

## Train Model

In [ ]:
down_sampling_factor = 5 # 5 minute window

## Evaluate Model